## Notebook to draw links between quotebank and our movie dataset
In this notebook we join our two datasets: quotes and movies.
Our first method simply draw links on the speaker of the quote: was the speaker part of the movie crew (as director/main actors/actrices).

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

We run the code separately for each year of interest.

In [15]:
year = "2020"
#open movie crew data set
df1 = pd.read_csv("../moviePreprocessing/movie_data_2015_2020.csv")
df1.head()
#remove some categories, if not we would have for e.g. Joe Biden as an actor because of an archive footage
df1 = df1[df1["category"]!="archive_footage"]
df1 = df1[df1["characters"]!="[\"Self\"]"]

In [16]:
def process_chunk(chunk):
    print(f'Processing chunk with {len(chunk)} rows')
    print(chunk.columns)
    #here you can filter etc
 
# read df chunk by chunk, long! 
with pd.read_json(f'../dataGitIgnore/quoteBank/quotes-{year}.json.bz2', lines=True, compression='bz2', chunksize=1000000) as df_reader:
    pbar = tqdm(enumerate(df_reader))
    for i,chunk in pbar:
        if i == 0:
            quotes = chunk[chunk["speaker"].isin(df1["primaryName"])]
        else:
            quotes = pd.concat((quotes,chunk[chunk["speaker"].isin(df1["primaryName"])]))
        pbar.set_description(f"Number of quotes {quotes.shape[0]}")

Number of quotes 137626: : 6it [04:33, 45.64s/it]


In [17]:
print(f"**{quotes.shape[0]} citations founds :)**")

**137626 citations founds :)**


In [18]:
quotes.head()

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
22,2020-04-01-000532,7pm is when most hospitals change shifts. That...,Amy Schumer,[Q4749380],2020-04-01 11:25:48,1,"[[Amy Schumer, 0.9294], [None, 0.0625], [Andy ...",[https://www.buzzfeed.com/benhenry/amy-schumer...,E
153,2020-02-03-006814,"As Graham [ Norton ] was saying, tonight is al...",Brad Pitt,"[Q35332, Q373912]",2020-02-03 10:38:47,1,"[[Brad Pitt, 0.3642], [None, 0.254], [Margot R...",[http://www.independent.co.uk/arts-entertainme...,E
211,2020-02-02-004706,Because this hardworking talented human of min...,Ashwiny Iyer Tiwari,[Q28012317],2020-02-02 05:14:15,2,"[[Ashwiny Iyer Tiwari, 0.4128], [None, 0.3853]...",[https://www.hindustantimes.com/bollywood/kang...,E
216,2020-01-29-010817,Beggin' On You Knees.,Halston Sage,[Q513169],2020-01-29 00:00:00,1,"[[Halston Sage, 0.7748], [None, 0.1437], [Zac ...",[https://hollywoodlife.com/feature/who-is-hals...,E
256,2020-04-06-006580,But it's definitely the peak of his NBA career.,Kobe Bryant,[Q25369],2020-04-06 01:03:01,1,"[[Kobe Bryant, 0.4337], [None, 0.2996], [Rob P...",[http://full.gulf-times.com/story/660128/Kobe-...,E


In [19]:
#save to the mergedData folder
quotes.to_csv(f"../dataGitIgnore/mergedData/movie_{year}_crew_quotes.csv.gz",compression='gzip')

In [14]:
#check
quotes.groupby("speaker").agg(count=("speaker","count")).sort_values(["count"],ascending=False)

,count
speaker,
Jimmy Kimmel,8421
Roy Moore,5747
John Fox,4282
Selena Gomez,3926
Kyrie Irving,3810
...,...
Ally Ioannides,1
Andrew Hollander,1
Gonzalo Maza,1
